<a href="https://colab.research.google.com/github/Anoif01/Sentiment-Analysis-FrozenLayer/blob/main/Git_Plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!pip install -U kaleido

In [ ]:
import plotly.graph_objects as go
import wandb
import pandas as pd
import numpy as np

In [ ]:
def norm(x):
    x = np.array(x)
    x_normalized = (x - x.min()) / (x.max() - x.min())
    return x_normalized

In [ ]:
api = wandb.Api()
runs = api.runs("id/project_name")

configs = []
summarys = []
names = []

for run in runs:
    print("Run ID:", run.id)
    print("Name:", run.name)
    print("---")

    configs.append(run.config)
    summarys.append(run.summary)
    names.append(run.name)

runs = None

In [ ]:
config_name = ['model_name', 'N', 'bs', 'lr','gpu']
summary_name = ['epoch', 'test_acc', 'mean_train_time', 'trainable_params', 'train_loss', 'val_accuracy']
merge_dicts = []
for i, config in enumerate(configs):
    filtered_config = {k: config[k] for k in config_name if k in config}
    filtered_summary = {k: summarys[i][k] for k in summary_name if k in summarys[i]}
    merge_dict = {**filtered_config, **filtered_summary}
    merge_dicts.append(merge_dict)

runs_df = pd.DataFrame(merge_dicts)

runs_df

In [ ]:
bert_df = runs_df[runs_df['model_name'] == 'bert-base-uncased'].sort_values(by='N').reset_index(drop=True)
tiny_df = runs_df[runs_df['model_name'] == 'prajjwal1/bert-tiny'].sort_values(by='N').reset_index(drop=True)
mini_df = runs_df[runs_df['model_name'] == 'prajjwal1/bert-mini'].sort_values(by='N').reset_index(drop=True)
small_df = runs_df[runs_df['model_name'] == 'prajjwal1/bert-small'].sort_values(by='N').reset_index(drop=True)
medium_df = runs_df[runs_df['model_name'] == 'prajjwal1/bert-medium'].sort_values(by='N').reset_index(drop=True)
distil_df = runs_df[runs_df['model_name'] == 'distilbert/distilbert-base-uncased'].sort_values(by='N').reset_index(drop=True)

In [ ]:
frac = 2/9 #1/4， 2/9
bert_df['trainable_params'] = bert_df['trainable_params'].apply(lambda x : np.power(x, frac))
tiny_df['trainable_params'] = tiny_df['trainable_params'].apply(lambda x : np.power(x, frac))
distil_df['trainable_params'] = distil_df['trainable_params'].apply(lambda x : np.power(x, frac))
mini_df['trainable_params'] = mini_df['trainable_params'].apply(lambda x : np.power(x, frac))
small_df['trainable_params'] = small_df['trainable_params'].apply(lambda x : np.power(x, frac))
medium_df['trainable_params'] = medium_df['trainable_params'].apply(lambda x : np.power(x, frac))

In [ ]:
fig = go.Figure()

color_dict = {
              'dark_red':['#C8A4C2','#BA7E92','#BA7E92','#9F3331','#920D00'],
              'red':['#D6AECB','#D792A3', '#D9757C','#DA5954','#DB3D2C'],
              'orange':['#DBBDCD','#E1B0A7', '#E6A381','#EC965B','#F28935'],
              'light_yellow':['#DBCEC2','#E0D192','#E6D561','#EBD831','#F1DC00'],
              'light_green':['#CDCACD', '#C4CAA8', '#BCCB82', '#B3CB5D','#ABCB37'],
              }

bert_name = "Bert_Base"
bert_color = color_dict['dark_red'][-1]
bert_color_scale = color_dict['dark_red']

distil_name = "Distil_Bert"
distil_color = color_dict['red'][-1]
distil_color_scale = color_dict['red']

small_name = "Bert_Small"
small_color = color_dict['orange'][-1]
small_color_scale = color_dict['orange']

mini_name = "Bert_Mini"
mini_color = color_dict['light_yellow'][-1]
mini_color_scale =  color_dict['light_yellow']

tiny_name = "Bert_Tiny"
tiny_color = color_dict['light_green'][-1]
tiny_color_scale = color_dict['light_green']

font_family = 'Arial'
marker_sym = 'circle'
mode = 'lines+markers'
opacity = 0.85
dash_line = 'dot'

fig.add_trace(go.Scatter(x=bert_df.N,
                         y=bert_df.test_acc,
                         name=bert_name,
                         mode=mode,
                         line=dict(color=bert_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=bert_df.trainable_params,
                                     color=norm([i*5 for i in range(bert_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, bert_color_scale[-1]] for i in range(5)],
                                     ),
                         ))

fig.add_trace(go.Scatter(x=distil_df.N,
                         y=distil_df.test_acc,
                         name=distil_name,
                         mode=mode,
                         line=dict(color=distil_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=distil_df.trainable_params,
                                     color=norm([i*5 for i in range(distil_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, distil_color_scale[-1]] for i in range(5)],
                                     )
                         ))

fig.add_trace(go.Scatter(x=small_df.N,
                         y=small_df.test_acc,
                         name=small_name,
                         mode=mode,
                         line=dict(color=small_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=small_df.trainable_params,
                                     color=norm([i*5 for i in range(small_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, small_color_scale[-1]] for i in range(5)],
                                     )
                         ))

fig.add_trace(go.Scatter(x=mini_df.N,
                         y=mini_df.test_acc,
                         name=mini_name,
                         mode=mode,
                         line=dict(color=mini_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=mini_df.trainable_params,
                                     color=norm([i*5 for i in range(mini_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, mini_color_scale[-1]] for i in range(5)],
                                     )
                         ))


fig.add_trace(go.Scatter(x=tiny_df.N,
                         y=tiny_df.test_acc,
                         name=tiny_name,
                         mode=mode,
                         line=dict(color=tiny_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=tiny_df.trainable_params,
                                     color=norm([i*5 for i in range(tiny_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, tiny_color_scale[-1]] for i in range(5)], #yellow
                                     )
                         ))



fig.update_layout(title='',
                  xaxis_title="Number of Frozen Layer",
                  yaxis_title="Test Accuracy",
                  legend_title="",
                  showlegend=False,
                  font=dict(family=font_family,
                            size=15,
                            color="#75767A")) #dark gray

# Title format
fig.update_layout(title_font_family=font_family,
                  title_font_color='WHITE',
                  title_font_size=18,
                  title_y=0.96,
                  title_x=0.5,
                  title_xanchor='center',
                  title_yanchor='top')

# Legend
fig.update_layout(legend_orientation='v', # h horizon/ v verticle
                  legend_y=0.05,
                  legend_x=0.9,
                  legend_xanchor='center',
                  legend_yanchor='bottom')
fig.update_layout(legend=dict(x=0.88,bgcolor="rgba(0,0,0,0)",bordercolor="rgba(0,0,0,0)",borderwidth=1)) #transparent

# X and Y axis
fig.update_layout(
    yaxis=dict(
        showgrid=True,
        showline=False,
        zeroline=False,
        gridcolor='#6D6E69',
        gridwidth=1,
        griddash='2px,10px'  # styles: 'solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot'
    ),
    xaxis=dict(
        showgrid=False,
        showline=False,
        zeroline=False,
        gridcolor='#666666',
        gridwidth=1,
        griddash='dot'
    )
)

# Modify the size of the layout and set the background color of the plotting area.
fig.update_layout(width=1200, height=700, margin=dict(l=75,r=50,b=50,t=50,),paper_bgcolor="#202020",) #2E2E2E 323234
fig.update_layout(yaxis=dict(range=[0.71, 0.96])) # Setting the Y-axis range
fig.update_layout(plot_bgcolor='#202020')

fig.show()

# Save the chart as a PNG file
fig.write_image('your_figure.png')

In [ ]:
fig = go.Figure()

color_dict = {
              'dark_red':['#C8A4C2','#BA7E92','#BA7E92','#9F3331','#920D00'],
              'red':['#D6AECB','#D792A3', '#D9757C','#DA5954','#DB3D2C'],
              'orange':['#DBBDCD','#E1B0A7', '#E6A381','#EC965B','#F28935'],
              'light_yellow':['#DBCEC2','#E0D192','#E6D561','#EBD831','#F1DC00'],
              'light_green':['#CDCACD', '#C4CAA8', '#BCCB82', '#B3CB5D','#ABCB37'],
              }

bert_name = "Bert_Base"
bert_color = color_dict['dark_red'][-1]
bert_color_scale = color_dict['dark_red']

distil_name = "Distil_Bert"
distil_color = color_dict['red'][-1]
distil_color_scale = color_dict['red']

small_name = "Bert_Small"
small_color = color_dict['orange'][-1]
small_color_scale = color_dict['orange']

mini_name = "Bert_Mini"
mini_color = color_dict['light_yellow'][-1]
mini_color_scale =  color_dict['light_yellow']

tiny_name = "Bert_Tiny"
tiny_color = color_dict['light_green'][-1]
tiny_color_scale = color_dict['light_green']

font_family = 'Arial'
marker_sym = 'diamond'
mode = 'lines+markers'
opacity = 0.85
dash_line = 'dot'

fig.add_trace(go.Scatter(x=bert_df.N,
                         y=bert_df.mean_train_time,
                         name=bert_name,
                         mode=mode,
                         line=dict(color=bert_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=bert_df.trainable_params,
                                     color=norm([i*5 for i in range(bert_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, bert_color_scale[-1]] for i in range(5)],
                                     ),
                         ))



fig.add_trace(go.Scatter(x=distil_df.N,
                         y=distil_df.mean_train_time,
                         name=distil_name,
                         mode=mode,
                         line=dict(color=distil_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=distil_df.trainable_params,
                                     color=norm([i*5 for i in range(distil_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, distil_color_scale[-1]] for i in range(5)],
                                     )
                         ))

fig.add_trace(go.Scatter(x=small_df.N,
                         y=small_df.mean_train_time,
                         name=small_name,
                         mode=mode,
                         line=dict(color=small_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=small_df.trainable_params,
                                     color=norm([i*5 for i in range(small_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, small_color_scale[-1]] for i in range(5)],
                                     )
                         ))


fig.add_trace(go.Scatter(x=mini_df.N,
                         y=mini_df.mean_train_time,
                         name=mini_name,
                         mode=mode,
                         line=dict(color=mini_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=mini_df.trainable_params,
                                     color=norm([i*5 for i in range(mini_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, mini_color_scale[-1]] for i in range(5)],
                                     )
                         ))


fig.add_trace(go.Scatter(x=tiny_df.N,
                         y=tiny_df.mean_train_time,
                         name=tiny_name,
                         mode=mode,
                         line=dict(color=tiny_color, width=2, dash=dash_line),
                         marker=dict(symbol=marker_sym,
                                     opacity=opacity,
                                     size=tiny_df.trainable_params,
                                     color=norm([i*5 for i in range(tiny_df.shape[0])][::-1]),
                                     colorscale=[[i*0.25, tiny_color_scale[-1]] for i in range(5)],
                                     )
                         ))

fig.add_trace(go.Scatter(x=bert_df.N[3:],
                         y=bert_df.mean_train_time[3:],
                         name=bert_name,
                         showlegend=False,
                         marker=dict(symbol=marker_sym,
                                     size=7,
                                     color=bert_color)
                         ))

fig.add_trace(go.Scatter(x=distil_df.N,
                         y=distil_df.mean_train_time,
                         name=distil_name,
                         showlegend=False,
                         marker=dict(symbol=marker_sym,
                                     size=7,
                                     color=distil_color
                                     )
                         ))
fig.add_trace(go.Scatter(x=small_df.N,
                         y=small_df.mean_train_time,
                         name=small_name,
                         showlegend=False,
                         marker=dict(symbol=marker_sym,
                                     size=7,
                                     color=small_color)
                         ))

fig.add_trace(go.Scatter(x=mini_df.N,
                         y=mini_df.mean_train_time,
                         name=mini_name,
                         showlegend=False,
                         marker=dict(symbol=marker_sym,
                                     size=5,
                                     color=mini_color)
                         ))
fig.add_trace(go.Scatter(x=tiny_df.N,
                         y=tiny_df.mean_train_time,
                         name=mini_name,
                         showlegend=False,
                         marker=dict(symbol=marker_sym,
                                     size=5,
                                     color=mini_color)
                         ))

fig.update_layout(title='',
                  xaxis_title="Number of Frozen Layer",
                  yaxis_title="Test Accuracy",
                  legend_title="",
                  showlegend=False,
                  font=dict(family=font_family,
                            size=15,
                            color="#75767A"))


fig.update_layout(title_font_family=font_family,
                  title_font_color='WHITE',
                  title_font_size=18,
                  title_y=0.96,
                  title_x=0.5,
                  title_xanchor='center',
                  title_yanchor='top')


fig.update_layout(legend_orientation='v', # h horizon/ v verticle
                  legend_y=0.05,
                  legend_x=0.9,
                  legend_xanchor='center',
                  legend_yanchor='bottom')

fig.update_layout(legend=dict(x=0.88,bgcolor="rgba(0,0,0,0)",bordercolor="rgba(0,0,0,0)",borderwidth=1))

# X and Y axis
fig.update_layout(
    yaxis=dict(
        showgrid=True,
        showline=False,
        zeroline=False,
        type='log',
        gridcolor='#6D6E69',
        gridwidth=1,
        griddash='2px,10px'
    ),
    xaxis=dict(
        showgrid=False,
        showline=False,
        zeroline=False,
        gridcolor='#666666',
        gridwidth=1,
        griddash='dot'
    )
)

fig.update_layout(width=1200, height=700, margin=dict(l=75,r=50,b=50,t=50,),paper_bgcolor="#202020",) #2E2E2E 323234
fig.update_layout(plot_bgcolor='#202020')

fig.show()

fig.write_image('your_figure2.png')
